<a href="https://www.kaggle.com/code/enricofindley/ultimate-music-analytics-challenge?scriptVersionId=143625730" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# The Ultimate Music Analytics Challenge

Welcome to the Ultimate Music Analytics Challenge! In this notebook, we will embark on a journey to explore the fascinating world of music using a rich dataset containing a wide range of attributes for songs. Let's dive in and discover some interesting insights or information about the music industry.

## Table of Contents
1. [Introduction](#introduction)
2. [Data Exploration](#data-exploration)
3. [Data Visualization](#data-visualization)
4. [Exploratory Data Analysis](#exploratory-data-analysis)
5. [Predictive Modeling](#predictive-modeling)
6. [Conclusion](#conclusion)

## Introduction <a id="introduction"></a>

Music is a universal language, and it evolves over time. With this dataset, we have the opportunity to analyze various aspects of songs, from their attributes like tempo and energy to their popularity on streaming platforms.

Let's start by loading the data and getting an overview of what's inside the data.


In [ ]:
# Import necessary libraries and load the dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Import necessary machine learning libraries and split the data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')
# Load the dataset
data = pd.read_csv('/kaggle/input/top-spotify-songs-2023/spotify-2023.csv', encoding = "ISO-8859-1")
# Filter the rows where 'track_title' is not equal to "Love Grows (Where My Rosemary Goes)"
data = data[data['track_name'] != "Love Grows (Where My Rosemary Goes)"]

# Reset the index after removing rows
data.reset_index(drop=True, inplace=True)

## Data Exploration <a id="data-exploration"></a>

Before we dive into the analysis, it's crucial to understand our data. Let's take a quick look at the first few rows and check for missing values.

In [ ]:
# Display the first few rows of the dataset
data.head()

In [ ]:
# Check for missing values
data.isnull().sum()

## Data Visualization <a id="data-visualization"></a>
They say a picture is worth a thousand words. Let's create some visually appealing plots and chart to better understand our dataset.

In [ ]:
plt.figure(figsize=(8, 8))
labels = ['Danceability', 'Valence', 'Energy']
sizes = [data['danceability_%'].iloc[0], data['valence_%'].iloc[0], data['energy_%'].iloc[0]]
colors = ['skyblue', 'lightcoral', 'lightgreen']
explode = (0.1, 0.1, 0.1)  # explode the 1st slice (Danceability)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Danceability, Valence, and Energy Distribution')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(x='key', data=data, palette='Set3')
plt.title('Key Distribution')

plt.subplot(1, 2, 2)
sns.countplot(x='mode', data=data, palette='Set2')
plt.title('Mode Distribution')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='danceability_%', y='valence_%', data=data, hue='mode', palette='Set2', s=100)
plt.title('Danceability vs. Valence')
plt.xlabel('Danceability %')
plt.ylabel('Valence %')
plt.legend(title='Mode')
plt.show()

In [ ]:
sns.set(style='whitegrid')
columns_to_plot = ['danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%']
sns.pairplot(data[columns_to_plot], palette='coolwarm')
plt.suptitle('Pairplot of Audio Attributes', y=1.02)
plt.show()

In [ ]:
# Exclude non-numeric columns (e.g., song names) from the correlation analysis
numeric_data = data.select_dtypes(include=[np.number])

# Compute the correlation matrix for numeric data
corr_matrix = numeric_data.corr()

# Now you can create the correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

## Exploratory Data Analysis <a id="exploratory-data-analysis"></a>
Now, let's dig deeper into the data and uncover interesting patterns.

In [ ]:
# Convert 'released_year' column to numeric (assuming it's originally in a string format)
data['released_year'] = data['released_year'].astype(int)

# Group data by released year and calculate the mean stream count
yearly_streams = data.groupby('released_year')['streams'].mean().reset_index()

# Plot the mean stream count over the years
plt.figure(figsize=(12, 6))
sns.lineplot(data=yearly_streams, x='released_year', y='streams')
plt.xlabel('Released Year')
plt.ylabel('Mean Streams')
plt.title('Mean Streams Over the Years')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(data['bpm'], bins=20, kde=True, color='skyblue')
plt.title('Distribution of BPM (Beats Per Minute)')
plt.xlabel('BPM')
plt.ylabel('Count')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.swarmplot(x='mode', y='danceability_%', data=data, palette='Set2')
plt.title('Danceability by Modes')
plt.xlabel('Mode')
plt.ylabel('Danceability %')
plt.show()

In [ ]:
sns.set(style='whitegrid')
columns_to_plot = ['danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%']
sns.pairplot(data, hue='mode', palette='Set2', vars=columns_to_plot)
plt.suptitle('Pairplot of Audio Attributes with Mode Hue', y=1.02)
plt.show()

## Predictive Modeling <a id="predictive-modeling"></a>
Can we predict song popularity based on certain attributes? Let's build a simple predictive model.

In [ ]:
# Define the number of folds (k)
k = 5  # You can adjust this value as needed

# Define your model
model = LinearRegression()

# Define the input features (X) and target variable (y)
X = data[['valence_%', 'energy_%']]
y = data['streams']

# Create a KFold cross-validation object
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Perform k-fold cross-validation and calculate RMSE for each fold
rmse_scores = np.sqrt(-cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error'))

# Print the RMSE for each fold
for i, rmse in enumerate(rmse_scores):
    print(f'Fold {i+1} RMSE: {rmse}')

# Calculate and print the mean RMSE across all folds
mean_rmse = np.mean(rmse_scores)
print(f'Mean RMSE: {mean_rmse}')

## Conclusion <a id="conclusion"></a>
Our journey through the world of music analytics has been enlightening. We've explored the data, visualized key trends, and even attempted to predict song popularity. The possibilities with this dataset are endless, and we've only scratched the surface.